In [8]:
import warnings
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
from dateutil import tz
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:99.5% !important;} </style>"))
%config Completer.use_jedi = False

In [10]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, Label, Range1d, BasicTickFormatter, LabelSet, DatetimeTickFormatter, DataRange1d, Span, ColorBar, Title
from bokeh.models.tools import HoverTool
from bokeh.layouts import layout, column, row
from bokeh.io import save
plot_width = 700
plot_height = 500

In [11]:
import matplotlib as mpl
rcparams = {'font.family':         'sans-serif',
            'font.sans-serif':     ['Helvetica'],
            'axes.labelsize':      28,
            'axes.titlesize':      28,
            'legend.fontsize':     20,
            'ytick.right':         'off',
            'xtick.top':           'off',
            'ytick.left':          'on',
            'xtick.bottom':        'on',
            'xtick.labelsize':     '28',
            'ytick.labelsize':     '28',
            'axes.linewidth':       2.5,
            'xtick.major.width':    1.8,
            'xtick.minor.width':    1.8,
            'xtick.major.size':     14,
            'xtick.minor.size':     7,
            'xtick.major.pad':      10,
            'xtick.minor.pad':      10,
            'ytick.major.width':    1.8,
            'ytick.minor.width':    1.8,
            'ytick.major.size':     14,
            'ytick.minor.size':     7,
            'ytick.major.pad':      10,
            'ytick.minor.pad':      10,
            'axes.labelpad':        15,
            'axes.titlepad':        15,
            'axes.spines.right':    False,
            'axes.spines.top':      False,
#             'axes.spines.left':      False
           }
mpl.rcParams.update(rcparams)

In [12]:
from_zone = tz.tzutc()
to_zone = tz.tzlocal()

# Data

In [14]:
df = pd.read_csv('data/scrobbles_ansesu.csv')
from_zone = tz.tzutc()
to_zone = tz.tzlocal()
df['date'] = [datetime.strptime(date, '%d %b %Y, %H:%M').replace(tzinfo=from_zone).astimezone(to_zone) for date in df.utc_time]
df['date']= pd.to_datetime(df['date'].values)
df = df.set_index('date')

for idx in df[df.artist == '水曜日のカンパネラ'].index:
    df.at[idx, 'artist'] = 'Wednesday Campanella'
for idx in df[df.artist == '水曜日のカンパネラ×shu uemura'].index:
    df.at[idx, 'artist'] = 'Wednesday Campanella'    
for idx in df[df.artist == 'Frederic'].index:
    df.at[idx, 'artist'] = 'フレデリック'    

In [15]:
year = 2019
# Create dataframe of specific year
df['year'] = [date.year for date in df.index]
df_year = df[df.year==2019].copy(deep=True)

# Create a column 'track - artist'
which_ = 'track'
l=[]
for i in df_year[f'{which_}'].values:
    artist = df[df[f'{which_}'] == i].artist.values[0]
    l.append(f'{i} - {artist}')
df_year[f'{which_}_w_artist'] =l

# Create a column 'album - artist'
which_ = 'album'
l=[]
for i in df_year[f'{which_}'].values:
    try:
        artist = df[df[f'{which_}'] == i].artist.values[0]
        l.append(f'{i} - {artist}')
    except:
        artist = ''
        l.append(f'{i} - {artist}')        
df_year[f'{which_}_w_artist'] =l

# Prepare data for flourish.io

## All-time artist

In [6]:
which_ = 'artist'
groupby_ = df.groupby(by=[df.index, which_]).agg('count')
groupby_.index.names = ['date', which_]
groupby_ = groupby_.reset_index()
min_year, min_month = groupby_.date.min().year, groupby_.date.min().month
max_year, max_month = groupby_.date.max().year, groupby_.date.max().month
f'starts at {min_month}/{min_year} and ends at {max_month}/{max_year}'

'starts at 7/2012 and ends at 7/2020'

In [7]:
start_date = datetime(min_year, min_month-1, 1)
end_date = datetime(max_year, max_month+1, 1)
date_range = pd.date_range(start_date, end_date, freq='M', closed='right').tolist()

In [8]:
all_artists = groupby_.artist.unique()
df_ = pd.DataFrame({'artist': all_artists})

for date1, date2 in zip(date_range[:-2], date_range[1:]):
    l=[]
    for artist in all_artists:
        mask = (groupby_.date >= date1) & (groupby_.date < date2) & (groupby_.artist==artist)
        l.append(groupby_[mask].track.sum())
    month = date1.month + 1
    year = date1.year
    if month == 13:
        month = 1
        year = year + 1
    df_[f'{month}/{year}']=l

l=[]
date1 = date_range[-2]
date2 = date_range[-1]
for artist in all_artists:
    mask = (groupby_.date >= date1) & (groupby_.date <= date2) & (groupby_.artist==artist)
    l.append(groupby_[mask].track.sum())
month = date1.month + 1
year = date1.year
if month == 13:
    month = 1
    year = year + 1
df_[f'{month}/{year}']=l

In [9]:
del df_['artist']
df_ = df_.cumsum(axis=1)
df_['artist'] = all_artists
df_.to_csv('ansesu_alltime.csv')

## Yearly artist

In [10]:
which_ = 'artist'
year = 2019
groupby_ = df.groupby(by=[df.index, which_]).agg('count')
groupby_.index.names = ['date', which_]
groupby_ = groupby_.reset_index()
groupby_['year'] = [date.year for date in groupby_.date]
groupby_= groupby_[groupby_['year']==year]

In [11]:
start_date = datetime(year, 1, 1)
end_date = datetime(year+1, 1, 2)
date_range = pd.date_range(start_date, end_date, freq='d', closed='left').tolist()

In [12]:
all_artists = groupby_.artist.unique()
df_ = pd.DataFrame({'artist': all_artists})

for date1, date2 in zip(date_range[:-2], date_range[1:]):
    l=[]
    for artist in all_artists:
        mask = (groupby_.date >= date1) & (groupby_.date < date2) & (groupby_.artist==artist)
        l.append(groupby_[mask].track.sum())
    day = date1.day    
    month = date1.month
    year = date1.year
    df_[f'{day}/{month}/{year}']=l
    
l=[]
date1 = date_range[-2]
date2 = date_range[-1]
for artist in all_artists:
    mask = (groupby_.date >= date1) & (groupby_.date <= date2) & (groupby_.artist==artist)
    l.append(groupby_[mask].track.sum())
day = date1.day 
month = date1.month
year = date1.year
df_[f'{day}/{month}/{year}']=l

In [13]:
del df_['artist']
df_ = df_.cumsum(axis=1)
df_['artist'] = all_artists
df_.to_csv('ansesu_artist_2019.csv')

## Yearly tracks

In [14]:
which_ = 'track'
year = 2019
groupby_ = df.groupby(by=[df.index, which_]).agg('count')
groupby_.index.names = ['date', which_]
groupby_ = groupby_.reset_index()
groupby_['year'] = [date.year for date in groupby_.date]
groupby_= groupby_[groupby_['year']==year]

In [15]:
start_date = datetime(year, 1, 1)
end_date = datetime(year+1, 1, 2)
date_range = pd.date_range(start_date, end_date, freq='d', closed='left').tolist()

In [16]:
all_ = groupby_[which_].unique()
df_ = pd.DataFrame({which_: all_})

for date1, date2 in zip(date_range[:-2], date_range[1:]):
    l=[]
    for i in all_:
        mask = (groupby_.date >= date1) & (groupby_.date < date2) & (groupby_[which_]==i)
        l.append(groupby_[mask].artist.sum())
    day = date1.day    
    month = date1.month
    year = date1.year
    df_[f'{day}/{month}/{year}']=l
    
l=[]
date1 = date_range[-2]
date2 = date_range[-1]
for i in all_:
    mask = (groupby_.date >= date1) & (groupby_.date <= date2) & (groupby_[which_]==i)
    l.append(groupby_[mask].artist.sum())
day = date1.day 
month = date1.month
year = date1.year
df_[f'{day}/{month}/{year}']=l

In [17]:
del df_[which_]
df_ = df_.cumsum(axis=1)
df_[which_] = all_
l = []
for i in df_[f'{which_}'].values:
    artist = df[df[f'{which_}'] == i].artist.values[0]
    l.append(f'{i} - {artist}')
df_['full_name'] = l
df_.to_csv(f'ansesu_{which_}_2019.csv')

## Yearly albums

In [18]:
which_ = 'album'
year = 2019
groupby_ = df.groupby(by=[df.index, which_]).agg('count')
groupby_.index.names = ['date', which_]
groupby_ = groupby_.reset_index()
groupby_['year'] = [date.year for date in groupby_.date]
groupby_= groupby_[groupby_['year']==year]

In [19]:
start_date = datetime(year, 1, 1)
end_date = datetime(year+1, 1, 2)
date_range = pd.date_range(start_date, end_date, freq='d', closed='left').tolist()

In [20]:
all_ = groupby_[which_].unique()
df_ = pd.DataFrame({which_: all_})

for date1, date2 in zip(date_range[:-2], date_range[1:]):
    l=[]
    for i in all_:
        mask = (groupby_.date >= date1) & (groupby_.date < date2) & (groupby_[which_]==i)
        l.append(groupby_[mask].artist.sum())
    day = date1.day    
    month = date1.month
    year = date1.year
    df_[f'{day}/{month}/{year}']=l
    
l=[]
date1 = date_range[-2]
date2 = date_range[-1]
for i in all_:
    mask = (groupby_.date >= date1) & (groupby_.date <= date2) & (groupby_[which_]==i)
    l.append(groupby_[mask].artist.sum())
day = date1.day 
month = date1.month
year = date1.year
df_[f'{day}/{month}/{year}']=l

In [21]:
del df_[which_]
df_ = df_.cumsum(axis=1)
df_[which_] = all_
l = []
for i in df_[f'{which_}'].values:
    artist = df[df[f'{which_}'] == i].artist.values[0]
    l.append(f'{i} - {artist}')
df_['full_name'] = l
df_.to_csv(f'ansesu_{which_}_2019.csv')

# My yearly top tracks

In [95]:
which_='track'
track, counts = np.unique(df_year[f'{which_}_w_artist'], return_counts=True)
df_track = pd.DataFrame({
           'track': track,
           'count': counts
           })

In [107]:
for j, i in enumerate(df_track.sort_values('count', ascending=False)[['track','count']].values[:10]):
    print(f'{j+1}) {i[0]} ({i[1]} plays)')

1) Pick U Up - Foster the People (159 plays)
2) Imagination - Foster the People (102 plays)
3) Style - Foster the People (65 plays)
4) Half of the Way - Vulfpeck (61 plays)
5) Wait For The Moment - Vulfpeck (57 plays)
6) Worst Nites - Foster the People (52 plays)
7) 1612 - Vulfpeck (51 plays)
8) Don't Stop (Color on the Walls) - Foster the People (49 plays)
9) Goats in Trees - Foster the People (49 plays)
10) Helena Beat - Foster the People (48 plays)


# My yearly top albums

In [117]:
which_='album'
album, counts = np.unique(df_year[f'{which_}_w_artist'], return_counts=True)
df_album = pd.DataFrame({
           'album': album,
           'count': counts
           })

In [118]:
for j, i in enumerate(df_album.sort_values('count', ascending=False)[['album','count']].values[:10]):
    print(f'{j+1}) {i[0]} ({i[1]} plays)')

1) Torches - Foster the People (418 plays)
2) Supermodel - Foster the People (339 plays)
3) Thrill of the Arts - Vulfpeck (312 plays)
4) Sacred Hearts Club - Foster the People (302 plays)
5) hill climber - Vulfpeck (286 plays)
6) The Beautiful Game - Vulfpeck (280 plays)
7) Saintmotelevision - Saint Motel (236 plays)
8) Pick U Up - Foster the People (216 plays)
9) Fugue State - Vulfpeck (213 plays)
10) Mr Finish Line - Vulfpeck (194 plays)


# Yearly listening time distribution

In [16]:
df_year['hour'] = [date.hour for date in df_year.index]
hour, count = np.unique(df_year['hour'], return_counts=True)
df_hour = pd.DataFrame({'hour': hour,
                        'count': count})

In [17]:
filename = 'daily_hours'

fig = figure(plot_width=plot_width, plot_height=plot_height, tools="", sizing_mode='stretch_both')
source = ColumnDataSource(df_hour)

bar = fig.vbar(x='hour', top='count', width=0.9, source=source, fill_color='#F8766D', line_width=0, alpha=0.85)

hover = HoverTool(renderers=[bar])
hover.tooltips=[
    ('Horário', '@hour'+':00h'),
    ('Quantidade', '@count{,}'),
]

fig.add_tools(hover)
fig.xaxis.ticker = source.data['hour']

fig.xaxis.formatter = BasicTickFormatter(use_scientific=False)

fig.yaxis.axis_label = 'Número de músicas'
fig.xaxis.axis_label = 'Horário, h'

fig.toolbar.logo = None
fig.toolbar_location = None
fig.y_range.start = 0
fig.x_range.start = -.5
fig.x_range.end = 23.5
fig.xaxis.axis_label_text_font_size = "14pt"
fig.yaxis.axis_label_text_font_size = "14pt"
fig.xaxis.major_label_text_font_size = "12pt"
fig.yaxis.major_label_text_font_size = "12pt"
fig.xaxis.axis_label_text_font_style = 'normal'
fig.yaxis.axis_label_text_font_style = 'normal'
fig.xgrid.grid_line_color = None
fig.ygrid.grid_line_color = None
output_file(f"html/{filename}_{year}.html")
save(fig,f"html/{filename}_{year}.html")

'/home/andre/github/lastfm/html/daily_hours_2019.html'

# Yearly month count distribution

In [18]:
df_year['month'] = [date.month for date in df_year.index]
month, count = np.unique(df_year['month'], return_counts=True)
df_month = pd.DataFrame({'month': month,
                        'count': count})
df_month['month_str'] = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
df_month['month_str_complete'] = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']

In [19]:
filename = 'month_counts'

fig = figure(plot_width=plot_width, plot_height=plot_height, tools="", sizing_mode='stretch_both')
source = ColumnDataSource(df_month)

bar = fig.vbar(x='month', top='count', width=0.9, source=source, fill_color='#756bb1', line_width=0, alpha=0.85)

hover = HoverTool(renderers=[bar])
hover.tooltips=[
    ('Mês', '@month_str_complete'),
    ('Quantidade', '@count{,}'),
]

fig.add_tools(hover)
fig.xaxis.ticker = source.data['month']
fig.xaxis.major_label_overrides = {int(i): month for i, month in zip(df_month['month'], df_month['month_str'])}

fig.yaxis.axis_label = 'Número de músicas'
fig.xaxis.axis_label = 'Mês'

fig.toolbar.logo = None
fig.toolbar_location = None
fig.y_range.start = 0
fig.x_range.start = .5
fig.x_range.end = 12.5
fig.xaxis.axis_label_text_font_size = "14pt"
fig.yaxis.axis_label_text_font_size = "14pt"
fig.xaxis.major_label_text_font_size = "12pt"
fig.yaxis.major_label_text_font_size = "12pt"
fig.xaxis.axis_label_text_font_style = 'normal'
fig.yaxis.axis_label_text_font_style = 'normal'
fig.xgrid.grid_line_color = None
fig.ygrid.grid_line_color = None
output_file(f"html/{filename}_{year}.html")
save(fig,f"html/{filename}_{year}.html")

'/home/andre/github/lastfm/html/month_counts_2019.html'

# Weekdays count distribution

In [20]:
df_year['weekday'] = [date.weekday() for date in df_year.index]
weekday, count = np.unique(df_year['weekday'], return_counts=True)
df_weekday = pd.DataFrame({'weekday': weekday,
                        'count': count})
df_weekday['weekday_str'] = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
df_weekday['weekday_str_complete'] = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [21]:
filename = 'weekday_counts'

fig = figure(plot_width=plot_width, plot_height=plot_height, tools="", sizing_mode='stretch_both')
source = ColumnDataSource(df_weekday)

bar = fig.vbar(x='weekday', top='count', width=0.9, source=source, fill_color='#2ca25f', line_width=0, alpha=0.85)

hover = HoverTool(renderers=[bar])
hover.tooltips=[
    ('Mês', '@weekday_str_complete'),
    ('Quantidade', '@count{,}'),
]

fig.add_tools(hover)
fig.xaxis.ticker = source.data['weekday']
fig.xaxis.major_label_overrides = {int(i): month for i, month in zip(df_weekday['weekday'], df_weekday['weekday_str'])}

fig.yaxis.axis_label = 'Número de músicas'
fig.xaxis.axis_label = 'Dia da semana'

fig.toolbar.logo = None
fig.toolbar_location = None
fig.y_range.start = 0
fig.x_range.start = -.5
fig.x_range.end = 6.5
fig.xaxis.axis_label_text_font_size = "14pt"
fig.yaxis.axis_label_text_font_size = "14pt"
fig.xaxis.major_label_text_font_size = "12pt"
fig.yaxis.major_label_text_font_size = "12pt"
fig.xaxis.axis_label_text_font_style = 'normal'
fig.yaxis.axis_label_text_font_style = 'normal'
fig.xgrid.grid_line_color = None
fig.ygrid.grid_line_color = None
output_file(f"html/{filename}_{year}.html")
save(fig,f"html/{filename}_{year}.html")

'/home/andre/github/lastfm/html/weekday_counts_2019.html'